In [27]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

artists_data_path = "../data/artists.jsonl"
sessions_data_path = "../data/sessions.jsonl"
track_storage_data_path = "../data/track_storage.jsonl"
tracks_data_path = "../data/tracks.jsonl"
users_data_path = "../data/users.jsonl"

artists_data_frame = pd.read_json(artists_data_path, lines=True)
sessions_data_frame = pd.read_json(sessions_data_path, lines=True)
tracks_data_frame = pd.read_json(tracks_data_path, lines=True)
users_data_frame = pd.read_json(users_data_path, lines=True)

# Scalenie danych

In [28]:
merged_data_frame = pd.merge(sessions_data_frame, tracks_data_frame, left_on="track_id",
                             right_on="id")
merged_data_frame = pd.merge(merged_data_frame, users_data_frame, left_on="user_id",
                             right_on="user_id")
merged_data_frame = pd.merge(merged_data_frame, artists_data_frame, left_on="id_artist",
                             right_on="id")
# rename column
merged_data_frame = merged_data_frame.rename(columns={"name_x": "track_name"})
merged_data_frame = merged_data_frame.rename(columns={"name_y": "artist_name"})
merged_data_frame.head(5)


,timestamp,user_id,track_id,event_type,session_id,id_x,track_name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_name,city,street,favourite_genres,premium_user,id_y,name,genres
0,2023-01-03 05:09:55.000,101,2PmGtDUyJIpYBEtI1hQIVp,PLAY,124,2PmGtDUyJIpYBEtI1hQIVp,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,Klara Herdzik,Szczecin,pl. Floriana 55/22,"[permanent wave, mandopop, funk]",False,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"
1,2023-02-08 21:41:02.507,103,2PmGtDUyJIpYBEtI1hQIVp,PLAY,151,2PmGtDUyJIpYBEtI1hQIVp,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,Szymon Zawal,Gdynia,al. Głogowa 14/10,"[filmi, regional mexican, folk]",False,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"
2,2023-02-12 01:17:14.946,132,2PmGtDUyJIpYBEtI1hQIVp,PLAY,544,2PmGtDUyJIpYBEtI1hQIVp,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"
3,2023-01-03 03:59:59.738,132,5yxYokipsWlpDCt4Th4VVc,PLAY,534,5yxYokipsWlpDCt4Th4VVc,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"
4,2023-03-07 14:28:25.702,132,5yxYokipsWlpDCt4Th4VVc,PLAY,547,5yxYokipsWlpDCt4Th4VVc,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,3dBVyJ7JuOMt4GE9607Qin,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]"


## Wyliczenie etykiety, czy dana piosenka w danej sesji została pominięta przez danego użytkownika

In [29]:
# delete rows with event_type other than PLAY and SKIP
merged_data_frame = merged_data_frame[merged_data_frame["event_type"].isin(["PLAY", "SKIP"])]

# if for one user_id and track_id and session_id there is only PLAY event, then the track was not skipped
merged_data_frame["skipped"] = merged_data_frame.groupby(["user_id", "track_id", "session_id"])[
  "event_type"].transform(lambda x: x != "PLAY").astype(bool)

# if there are 2 rows with the same user_id, track_id and session_id, and different "skipped" value, remove the row with "skipped" = 0
merged_data_frame = merged_data_frame.drop_duplicates(subset=["user_id", "track_id", "session_id"],
                                                      keep="last")



## Dodanie nowych atrybutów

In [30]:
merged_data_frame["number_of_matching_genres"] = merged_data_frame.apply(lambda x: len(set(x["genres"]).intersection(set(x["favourite_genres"]))), axis=1)
merged_data_frame["month"] = merged_data_frame["timestamp"].dt.month
merged_data_frame["day_of_week"] = merged_data_frame["timestamp"].dt.dayofweek
merged_data_frame["hour_of_day"] = merged_data_frame["timestamp"].dt.hour
merged_data_frame["genres_with_favourite_genres"] = merged_data_frame["genres"] + merged_data_frame["favourite_genres"]
merged_data_frame["common_genres"] = merged_data_frame["genres_with_favourite_genres"].apply(lambda x: list(set(x)))

## Usunięcie niepotrzebnych kolumn

In [31]:
merged_data_frame = merged_data_frame.drop(columns=["id_x", "id_y", "event_type"])
merged_data_frame.head(10)

,timestamp,user_id,track_id,session_id,track_name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_name,city,street,favourite_genres,premium_user,name,genres,skipped,number_of_matching_genres,month,day_of_week,hour_of_day,genres_with_favourite_genres,common_genres
0,2023-01-03 05:09:55.000,101,2PmGtDUyJIpYBEtI1hQIVp,124,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,Klara Herdzik,Szczecin,pl. Floriana 55/22,"[permanent wave, mandopop, funk]",False,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False,0,1,1,5,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock, permanent wave, mandopop, funk]","[classic rock, folk rock, protopunk, permanent wave, mandopop, psychedelic rock, rock, funk, glam rock, album rock, art rock]"
1,2023-02-08 21:41:02.507,103,2PmGtDUyJIpYBEtI1hQIVp,151,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,Szymon Zawal,Gdynia,al. Głogowa 14/10,"[filmi, regional mexican, folk]",False,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False,0,2,2,21,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock, filmi, regional mexican, folk]","[classic rock, folk rock, protopunk, psychedelic rock, rock, folk, filmi, glam rock, album rock, regional mexican, art rock]"
2,2023-02-12 01:17:14.946,132,2PmGtDUyJIpYBEtI1hQIVp,544,Ballrooms Of Mars,34,247707,0,3dBVyJ7JuOMt4GE9607Qin,1972-07-21,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False,1,2,6,1,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock, psychedelic rock, country rock, rock en espanol]","[classic rock, folk rock, protopunk, country rock, psychedelic rock, rock, rock en espanol, glam rock, album rock, art rock]"
3,2023-01-03 03:59:59.738,132,5yxYokipsWlpDCt4Th4VVc,534,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False,1,1,1,3,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock, psychedelic rock, country rock, rock en espanol]","[classic rock, folk rock, protopunk, country rock, psychedelic rock, rock, rock en espanol, glam rock, album rock, art rock]"
4,2023-03-07 14:28:25.702,132,5yxYokipsWlpDCt4Th4VVc,547,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,Cezary Nóżka,Wrocław,aleja Olchowa 16,"[psychedelic rock, country rock, rock en espanol]",True,T. Rex,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock]",False,1,3,1,14,"[album rock, art rock, classic rock, folk rock, glam rock, protopunk, psychedelic rock, rock, psychedelic rock, country rock, rock en espanol]","[classic rock, folk rock, protopunk, country rock, psychedelic rock, rock, rock en espanol, glam rock, album rock, art rock]"
5,2023-03-10 04:07:21.284,107,5yxYokipsWlpDCt4Th4VVc,209,Solid Gold Easy Action,35,140067,0,3dBVyJ7JuOMt4GE9607Qin,1973-03-16,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,Kajetan Damps,Kraków,aleja Lazurowa 807,"[latin alternative, alternative rock, classic rock]",True,T. Rex,"[album rock, art rock, c

In [32]:
merged_data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9881 entries, 0 to 16822
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   timestamp                     9881 non-null   datetime64[ns]
 1   user_id                       9881 non-null   int64         
 2   track_id                      9881 non-null   object        
 3   session_id                    9881 non-null   int64         
 4   track_name                    9881 non-null   object        
 5   popularity                    9881 non-null   int64         
 6   duration_ms                   9881 non-null   int64         
 7   explicit                      9881 non-null   int64         
 8   id_artist                     9881 non-null   object        
 9   release_date                  9881 non-null   object        
 10  danceability                  9881 non-null   float64       
 11  energy                        9881

In [33]:
merged_data_frame.describe()

,timestamp,user_id,session_id,popularity,duration_ms,explicit,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,number_of_matching_genres,month,day_of_week,hour_of_day
count,9881,9881.000000,9881.000000,9881.000000,9.881000e+03,9881.000000,9881.000000,9881.000000,9881.00000,9881.000000,9881.000000,9881.000000,9881.000000,9881.000000,9881.000000,9881.000000,9881.000000,9881.00000,9881.000000,9881.000000
mean,2023-02-13 08:00:47.085888512,123.952434,427.961340,44.646594,2.310971e+05,0.080963,0.563325,0.600228,5.25463,-9.125568,0.083663,0.328242,0.067860,0.207252,0.539190,120.525376,0.147860,1.97227,2.834025,11.791317
min,2023-01-01 06:02:13.368000,101.000000,124.000000,0.000000,3.100000e+04,0.000000,0.064000,0.000592,0.00000,-35.898000,0.022700,0.000001,0.000000,0.016000,0.002280,36.241000,0.000000,1.00000,0.000000,0.000000
25%,2023-01-23 02:56:21.670000128,110.000000,248.000000,25.000000,1.803480e+05,0.000000,0.453000,0.428000,2.00000,-11.515000,0.033500,0.042500,0.000000,0.095100,0.338000,98.231000,0.000000,1.00000,1.000000,6.000000
50%,2023-02-13 19:17:18.136999936,125.000000,443.000000,39.000000,2.158000e+05,0.000000,0.570000,0.624000,5.00000,-8.446000,0.042800,0.223000,0.000044,0.127000,0.544000,120.031000,0.000000,2.00000,3.000000,12.000000
75%,2023-03-07 09:04:16.190000128,136.000000,585.000000,75.000000,2.626530e+05,0.000000,0.683000,0.794000,9.00000,-6.114000,0.077100,0.579000,0.005030,0.251000,0.742000,138.023000,0.000000,3.00000,5.000000,18.000000
max,2023-03-28 15:10:21.062000,150.000000,768.000000,96.000000,2.365307e+06,1.000000,0.975000,0.998000,11.00000,1.716000,0.965000,0.996000,0.978000,0.997000,0.991000,209.231000,3.000000,3.00000,6.000000,23.000000
std,NaN,14.826553,192.472969,25.085211,9.345541e+04,0.272793,0.165085,0.230087,3.55038,4.286563,0.121603,0.311197,0.199239,0.192774,0.250657,28.640296,0.375486,0.82167,1.969859,6.893432


## Współczynnik informacji wzajemnej

Sprawdzamy, czy atrybuty niosą ze sobą informację o zmiennej celu.

In [34]:
from sklearn import metrics
from scipy.stats import entropy

print("Entropy of skipped column: ", str(round(entropy(merged_data_frame["skipped"].value_counts()), 4)))

df = merged_data_frame.copy()
df["favourite_genres"] = df["favourite_genres"].apply(lambda x: str(x))
df["genres"] = df["genres"].apply(lambda x: str(x))
df["genres_with_favourite_genres"] = df["genres_with_favourite_genres"].apply(lambda x: str(x))
df["common_genres"] = df["common_genres"].apply(lambda x: str(x))

mutual_info_score = {}

for column in df.columns:
  if column != "skipped":
    mutual_info_score[column] = round(metrics.mutual_info_score(df["skipped"], df[column]), 4)

# print sorted by value
for key, value in sorted(mutual_info_score.items(), key=lambda item: item[1], reverse=True):
  print("%s: %s" % (key, value))

Entropy of skipped column:  0.657
genres_with_favourite_genres: 0.4927
common_genres: 0.4924
timestamp: 0.4805
track_id: 0.3277
track_name: 0.3237
tempo: 0.3164
duration_ms: 0.2954
loudness: 0.2777
id_artist: 0.2023
name: 0.2021
genres: 0.1645
release_date: 0.1609
instrumentalness: 0.1544
session_id: 0.1316
acousticness: 0.1277
user_id: 0.0972
artist_name: 0.0972
street: 0.0972
favourite_genres: 0.0972
liveness: 0.0732
valence: 0.0657
speechiness: 0.0607
energy: 0.0591
danceability: 0.0522
city: 0.031
premium_user: 0.0109
popularity: 0.0062
hour_of_day: 0.0044
day_of_week: 0.0016
key: 0.0006
month: 0.0004
explicit: 0.0002
number_of_matching_genres: 0.0001


/Users/rasztabigab/anaconda3/envs/IUM/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received binary values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/Users/rasztabigab/anaconda3/envs/IUM/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received binary values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/Users/rasztabigab/anaconda3/envs/IUM/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received binary values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/Users/rasztabigab/anaconda3/envs/IUM/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received binary v

## Macierz korelacji

In [35]:
df = df.drop(columns=["genres", "favourite_genres", "genres_with_favourite_genres", "common_genres", "track_id", "track_name", "id_artist", "release_date", "artist_name", "city", "street", "name"])

df["skipped"] = df["skipped"].astype(int)
df["premium_user"] = df["premium_user"].astype(int)
df["timestamp"] = pd.to_numeric(df["timestamp"], errors="coerce")

correlation_matrix = df.corrwith(df["skipped"])
correlation_matrix

timestamp                   -0.037091
user_id                     -0.010792
session_id                  -0.011029
popularity                   0.049451
duration_ms                  0.056545
explicit                     0.019890
danceability                -0.085718
energy                       0.015603
key                         -0.008343
loudness                     0.013799
speechiness                 -0.027947
acousticness                -0.023914
instrumentalness             0.026510
liveness                     0.016812
valence                     -0.067857
tempo                        0.004878
premium_user                 0.147678
skipped                      1.000000
number_of_matching_genres    0.002574
month                       -0.024181
day_of_week                 -0.013223
hour_of_day                  0.013549
dtype: float64

In [36]:
# save to jsonl file
merged_data_frame.to_json("../data/merged_data.jsonl", orient="records", lines=True)


## Podsumowanie

Jak widać na podstawie współczyników informacji wzajemnej, połączenie atrybutów "genres" i "favourite_genres" jest dobrym atrybutem do uczenia naszego modelu predykcyjnego.
Innymi dobrymi atrybutami są: "tempo", "duration_ms" i "loudness".

Początkowo zdecydowaliśmy się na użycie atrybutów "genres" i "favourite_genres" jako danych wejściowych modelu. Zmienną celu jest etykieta "skipped".
